# A\* Route Planner Using Advanced Data Structures.
We will implement A\* search to implement a "Google-maps" style route planning algorithm.  

### PathPlanner class

`__init__` - We initialize our path planner with a map, M, and typically a start and goal node. If either of these are `None`, the rest of the variables here are also set to none. 
- `closedSet` includes any explored/visited nodes. 
- `openSet` are any nodes on our frontier for potential future exploration. 
- `cameFrom` will hold the previous node that best reaches a given node
- `gScore` is the `g` in our `f = g + h` equation, or the actual cost to reach our current node
- `fScore` is the combination of `g` and `h`, i.e. the `gScore` plus a heuristic; total cost to reach the goal
- `path` comes from the `run_search` function..

`reconstruct_path` - This function just rebuilds the path after search is run, going from the goal node backwards using each node's `cameFrom` information.

`_reset` - Resets *most* of our initialized variables for PathPlanner. This *does not* reset the map, start or goal variables.

`run_search` -  The method checks whether the map, goal and start have been added to the class. Then, it will also check if the other variables, other than `path` are initialized (note that these are only needed to be re-run if the goal or start were not originally given when initializing the class.

`is_open_empty`, is used to check whether there are still nodes to explore. If we're at our goal, we reconstruct the path. If not, we move our current node from the frontier (`openSet`) and into explored (`closedSet`). Then, we check out the neighbors of the current node, check out their costs, and plan our next move.

## The Map

In [1]:

from helpers import Map, load_map_10, load_map_40, show_map
import math

%load_ext autoreload
%autoreload 2

### Map Basics

In [2]:
map_10 = load_map_10()
show_map(map_10)


The map above  shows a disconnected network of 10 intersections. The two intersections on the left are connected to each other but they are not connected to the rest of the road network. This map is quite literal in its expression of distance and connectivity. On the graph above, the edge between 2 nodes(intersections) represents a literal straight road not just an abstract connection of 2 cities.

These `Map` objects have two properties we will use to implement A\* search: `intersections` and `roads`

**Intersections**

The `intersections` are represented as a dictionary. 

In this example, there are 10 intersections, each identified by an x,y coordinate. The coordinates are listed below. You can hover over each dot in the map above to see the intersection number.

In [3]:
map_10.intersections

{0: (0.7798606835438107, 0.6922727646627362),
 1: (0.7647837074641568, 0.3252670836724646),
 2: (0.7155217893995438, 0.20026498027300055),
 3: (0.7076566826610747, 0.3278339270610988),
 4: (0.8325506249953353, 0.02310946309985762),
 5: (0.49016747075266875, 0.5464878695400415),
 6: (0.8820353070895344, 0.6791919587749445),
 7: (0.46247219371675075, 0.6258061621642713),
 8: (0.11622158839385677, 0.11236327488812581),
 9: (0.1285377678230034, 0.3285840695698353)}

**Roads**

The `roads` property is a list where `roads[i]` contains a list of the intersections that intersection `i` connects to.

In [4]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 
start=5
coords=map_10.intersections[start]
x=coords[0]
y=coords[1]
print(x)
print(y)

0.49016747075266875
0.5464878695400415


In [5]:
# This shows the full connectivity of the map
map_10.roads
len(map_10.intersections)

10

In [6]:
# map_40 is a bigger map than map_10
map_40 = load_map_40()
show_map(map_40)

### Advanced Visualizations

The map above shows a network of roads which spans 40 different intersections (labeled 0 through 39). 

The `show_map` function which generated this map also takes a few optional parameters which might be useful for visualizing the output of the search algorithm you will write.

* `start` - The "start" node for the search algorithm.
* `goal`  - The "goal" node.
* `path`  - An array of integers which corresponds to a valid sequence of intersection visits on the map.

In [7]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

### Pathplanner Class

In [8]:
import math
import heapq

class PathPlanner():
    """Construct a PathPlanner Object"""
    def __init__(self, M, start=None, goal=None):
        """ """
        self.map = M
        self.start= start
        self.goal = goal
        self.closedSet = self.create_closedSet() if goal != None and start != None else None
        self.openSet = self.create_openSet() if goal != None and start != None else None
        self.cameFrom = self.create_cameFrom() if goal != None and start != None else None
        self.gScore = self.create_gScore() if goal != None and start != None else None
        self.fScore = self.create_fScore() if goal != None and start != None else None
        self.path = self.run_search() if self.map and self.start != None and self.goal != None else None
    def reconstruct_path(self, current):
        """ Reconstructs path after search """
        total_path = [current]
        while current in self.cameFrom.keys():
            current = self.cameFrom[current]
            total_path.append(current)
        return total_path
    
    def _reset(self):
        """Private method used to reset the closedSet, openSet, cameFrom, gScore, fScore, and path attributes"""
        self.closedSet = None
        self.openSet = None
        self.cameFrom = None
        self.gScore = None
        self.fScore = None
        self.path = self.run_search() if self.map and self.start and self.goal else None

    def run_search(self):
        """ """
        if self.map == None:
            raise(ValueError, "Must create map before running search. Try running PathPlanner.set_map(start_node)")
        if self.goal == None:
            raise(ValueError, "Must create goal node before running search. Try running PathPlanner.set_goal(start_node)")
        if self.start == None:
            raise(ValueError, "Must create start node before running search. Try running PathPlanner.set_start(start_node)")

        self.closedSet = self.closedSet if self.closedSet != None else self.create_closedSet()
        self.openSet = self.openSet if self.openSet != None else  self.create_openSet()
        self.cameFrom = self.cameFrom if self.cameFrom != None else  self.create_cameFrom()
        self.gScore = self.gScore if self.gScore != None else  self.create_gScore()
        self.fScore = self.fScore if self.fScore != None else  self.create_fScore()

        while not self.is_open_empty():
            
            current = self.get_current_node()

            if current == self.goal:
                self.path = [x for x in reversed(self.reconstruct_path(current))]
                return self.path
            else:
                self.openSet.remove(current)
                self.closedSet.add(current)

            for neighbor in self.get_neighbors(current):
                if neighbor in self.closedSet:
                    continue    # Ignore the neighbor which is already evaluated.

                if not neighbor in self.openSet:    # Discover a new node
                    self.openSet.add(neighbor)
                    heapq.heappush(self.openHeap, (self.get_tentative_gScore(current, neighbor) ,neighbor))

                
                # The distance from start to a neighbor
                #the "dist_between" function may vary as per the solution requirements.
                if self.get_tentative_gScore(current, neighbor) >= self.get_gScore(neighbor):
                    continue        # This is not a better path.

                # This path is the best until now. Record it!
                self.record_best_path_to(current, neighbor)
        print("No Path Found")
        self.path = None
        return False
 
    def create_closedSet(self):
        """ Creates and returns a data structure suitable to hold the set of nodes already evaluated"""
        return set()

    
    def create_openSet(self):
        """ Creates and returns a data structure suitable to hold the set of currently discovered nodes 
        that are not evaluated yet. Initially, only the start node is known."""
        if self.start != None:
            self.openHeap=[]
            heapq.heappush(self.openHeap, (0, self.start)) 
            return set([self.start])
        raise(ValueError, "Must create start node before creating an open set. Try running PathPlanner.set_start(start_node)") 


    def create_cameFrom(self):
        """Creates and returns a data structure that shows which node can most efficiently be reached from another,
        for each node."""

        cameFrom = {}
        return cameFrom


    def create_gScore(self):
        """Creates and returns a data structure that holds the cost of getting from the start node to that node, 
        for each node. The cost of going from start to start is zero."""
        g_scores=[ float("infinity") for _ in range(len(self.map.intersections))]
        g_scores[self.start]=0.0
        return g_scores


    def create_fScore(self):
        """Creates and returns a data structure that holds the total cost of getting from the start node to the goal
        by passing by that node, for each node. That value is partly known, partly heuristic.
        For the first node, that value is completely heuristic."""
        f_scores=[ float("infinity") for _ in range(len(self.map.intersections))]
        f_scores[self.start]= 1000
        return f_scores

    def set_map(self, M):
        """Method used to set map attribute """
        self._reset(self)
        self.start = None
        self.goal = None
        self.map=M

    def set_start(self, start):
        """Method used to set start attribute """
        self._reset(self)
        self.start=start

    def set_goal(self, goal):
        """Method used to set goal attribute """
        self._reset(self)
        self.goal=goal
        # TODO: Set goal value. 

    def is_open_empty(self):
        """returns True if the open set is empty. False otherwise. """
        # TODO: Return True if the open set is empty. False otherwise.
        if len(self.openSet):
            return False
        else:
            return True

    def get_current_node(self):
        """ Returns the node in the open set with the lowest value of f(node)."""
        #cost, node = heapq.heappop(self.openHeap)
        return heapq.heappop(self.openHeap)[1] 
    # inefficient:
    #     node_scores=[] 
    #     openList = list(self.openSet) #to enable indexing
    #     for node in openList:   
    #         node_scores.append(self.calculate_fscore(node))
    #     min_index=node_scores.index(min(node_scores))
    #     print("self.openHeap",self.openHeap)
    #     print(node_scores)
    #     return openList[min_index]


    def get_neighbors(self, node):
        """Returns the neighbors of a node"""
        return self.map.roads[node] 

    def get_gScore(self, node):
        """Returns the g Score of a node"""
        return self.gScore[node]

    def distance(self, node_1, node_2):
        """ Computes the Euclidean L2 Distance"""
        node1_coords=self.map.intersections[node_1]
        node2_coords=self.map.intersections[node_2]
        return math.sqrt((node1_coords[0]-node2_coords[0])**2+ (node1_coords[1]-node2_coords[1])**2)

    def get_tentative_gScore(self, current, neighbor):
        """Returns the tentative g Score of a node + distance from the current node to it's neighbors"""
        return self.gScore[current]+self.distance(current, neighbor)

    def heuristic_cost_estimate(self, node):
        """ Returns the heuristic cost estimate of a node """
        if self.goal != None:
            return self.distance(node, self.goal)
        raise(ValueError, "Must create goal node before.")


    def calculate_fscore(self, node):
        """Calculate the f score of a node.F = G + H """
        return self.get_gScore(node)+self.heuristic_cost_estimate(node)

    def record_best_path_to(self, current, neighbor):
        """Record the best path to a node by updating cameFrom, gScore, and fScore   """
        self.cameFrom[neighbor]=current
        self.gScore[neighbor]=self.get_tentative_gScore(current, neighbor)
        self.fScore[neighbor]=self.get_gScore(neighbor)+ self.heuristic_cost_estimate(neighbor)
        #Reference:https://en.wikipedia.org/wiki/A*_search_algorithm



#### Visualize

Let's visualize the results of the algorithm!

In [9]:
# Visualize your the result of the above test! You can also change start and goal here to check other paths
start = 5
goal = 34

show_map(map_40, start=start, goal=goal, path=PathPlanner(map_40, start, goal).path)

In [10]:
from test import test

test(PathPlanner)

All tests pass! Congratulations!


In [12]:
# Visualize your the result of the above test! You can also change start and goal here to check other paths
start = 5
goal = 35

show_map(map_40, start=start, goal=goal, path=PathPlanner(map_40, start, goal).path)

## Questions

**Instructions**  

---
How would you explain A-Star to a family member(layman)?

**ANSWER**: A star is an algorithm that can find the closest path between two destinations. It incorporates information about explored paths and heuristics to find the minimum path. It's commonly used in games and map apps on your phone.


---
How does A-Star search algorithm differ from Uniform cost search? What about Best First search?

**ANSWER**:  Uniform cost search is uninformed and expands like a distance contour. Best first search and A* search are informed: they use a heuristic to go towards a goal. Best first search is greedy and expands towards nodes that are closest to the goal.The computed path may not be the shortest. On the other hand, A* expands towards the nodes that are closer to the goal while also optimizing for minimum path.



---
What is a heuristic?

**ANSWER**: Heuristic is simply an educated guess for solving problems.It can provide an approximate solution, but doesn't guarantee optimality or completeness.

---
What is a consistent heuristic?

**ANSWER**:A consistent heuristic assigns a cost to a state which is less than or equal to the cost of reaching a successor state plus the estimated cost of reaching the goal from that successor. A consistent heuristic implies that the cost between a state and its successor can't be overestimated.

---
What is a admissible heuristic? 

**ANSWER**: It's a heuristic where the cost of going from a state to the goal state is less than the true cost. It never overestimates the cost of reaching the goal. For example, a consistent heuristic would guarantee admissability since  the cost between a state and its successor can't be overestimated.





 ___ admissible heuristic are consistent.

*CHOOSE ONE*
    - All
    - Some
    - None
    
**ANSWER**: Some admissible heuristics are consistent.

---
___ consistent heuristic are admissible.

*CHOOSE ONE*
    - All
    - Some
    - None
    
**ANSWER**:All consistent heuristic are admissible.

---